In [0]:
from functools import reduce
from pyspark.sql import DataFrame

In [0]:
itens_cesta_basica = [
    {'nm_produto': 'Arroz', 'nm_tipo': '1 KG', 'vl_quantidade': 2},
    {'nm_produto': 'Feijao', 'nm_tipo': '1 KG', 'vl_quantidade': 1},
    {'nm_produto': 'Trigo', 'nm_tipo': '1 KG', 'vl_quantidade': 1},
    {'nm_produto': 'Acucar', 'nm_tipo': '1 KG', 'vl_quantidade': 1},
    {'nm_produto': 'Sardinha', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Amido de Milho', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Cafe', 'nm_tipo': '500', 'vl_quantidade': 1},
    {'nm_produto': 'Bolacha', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Fuba', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Milho', 'nm_tipo': '170', 'vl_quantidade': 1},
    {'nm_produto': 'Espaguete', 'nm_tipo': '500', 'vl_quantidade': 1},
    {'nm_produto': 'Molho de tomate', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Oleo de soja', 'nm_tipo': '900', 'vl_quantidade': 1},
    {'nm_produto': 'Tempero', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Gelatina', 'nm_tipo': '', 'vl_quantidade': 1},
    {'nm_produto': 'Refresco', 'nm_tipo': '', 'vl_quantidade': 2},
]

## Analise do tipo: Onde o item é mais barato?

In [0]:

def get_sql_query_item(nm_produto: str, nm_tipo: str, vl_quantidade: int) -> str:
    sql_template = ("""
    SELECT
            produtos.nm_produto,
            produtos.nm_tipo,
            precos.nm_rede, 
            precos.nm_bairro,
            precos.dt_ultima_atualizacao,
            precos.nm_tipo_preco_varejo,
            {vl_quantidade} AS vl_quantidade,
            precos.vl_preco_atacado AS vl_preco_unitario,
            ({vl_quantidade} * precos.vl_preco_atacado) AS vl_subtotal
        FROM precos_pmc.gold.f_precos precos
        LEFT JOIN precos_pmc.gold.d_produtos produtos ON produtos.cd_produto = precos.cd_produto 
        WHERE 1=1
            AND precos.dt_ultima_atualizacao >= DATE_SUB(NOW(), 15)
            AND produtos.nm_produto LIKE '%{nm_produto}%'
            AND nm_tipo LIKE '%{nm_tipo}%'
        ORDER BY 
            precos.vl_preco_atacado ASC
        LIMIT 1             
    """)

    return sql_template.format(
        nm_produto=nm_produto.upper(), 
        nm_tipo=nm_tipo.upper(), 
        vl_quantidade=vl_quantidade
    )



In [0]:

dfs = []
for item in itens_cesta_basica:
    df = spark.sql(get_sql_query_item(**item))
    
    if df.count() == 0:
        print('Não encontrado', item)

    dfs.append(df)

df = reduce(DataFrame.union, dfs)
df = df.orderBy('vl_subtotal').withColumn('vl_total', F.sum('vl_subtotal').over(Window.orderBy('vl_subtotal')))
display(df)


## Análise do tipo: Onde a cesta é mais barata?

In [0]:
def get_sql_query_basket(basket: list[dict]) -> str:
    sql_template = ("""
        SELECT
            '{nm_produto}' AS nm_pesquisa,
            produtos.nm_produto,
            produtos.nm_tipo,
            precos.nm_rede, 
            precos.nm_bairro,
            precos.dt_ultima_atualizacao,
            precos.nm_tipo_preco_varejo,
            {vl_quantidade} AS vl_quantidade,
            precos.vl_preco_atacado AS vl_preco_unitario,
            ({vl_quantidade} * precos.vl_preco_atacado) AS vl_subtotal
        FROM precos_pmc.gold.f_precos precos
        LEFT JOIN precos_pmc.gold.d_produtos produtos ON produtos.cd_produto = precos.cd_produto 
        WHERE 1=1
            AND precos.dt_ultima_atualizacao >= DATE_SUB(NOW(), 15)
            AND produtos.nm_produto LIKE '%{nm_produto}%'
            AND nm_tipo LIKE '%{nm_tipo}%'
      
    """)

    query = ''
    for item in basket:
        sql_item = sql_template.format(
            nm_produto=item.get('nm_produto').upper(), 
            nm_tipo=item.get('nm_tipo').upper(), 
            vl_quantidade=item.get('vl_quantidade')
        )

        query += sql_item
        query += ' \n UNION ALL \n'

    return query.rstrip(' \n UNION ALL \n')

In [0]:

sql_basket = get_sql_query_basket(itens_cesta_basica)
df_basket = spark.sql(sql_basket)
display(df_basket)


In [0]:

df_basket.createOrReplaceTempView('basket')
df_basket_result = spark.sql(f"""
    WITH 
        rank_precos AS
        (
            SELECT
                nm_rede,
                nm_bairro,
                nm_pesquisa,
                nm_produto,
                nm_tipo,
                vl_preco_unitario,
                vl_quantidade,
                dense_rank() OVER (PARTITION BY nm_rede, nm_bairro, nm_pesquisa ORDER BY vl_preco_unitario ASC) AS rank
            FROM basket
        ), mais_baratos AS 
    (
        SELECT
            nm_rede,
            nm_bairro,
            nm_pesquisa,
            nm_produto,
            nm_tipo,
            vl_preco_unitario,
            vl_quantidade,
            rank,
            COUNT(*) OVER (PARTITION BY nm_rede, nm_bairro) AS qtd_itens
        FROM rank_precos

    )
    SELECT
        nm_pesquisa,
        (nm_rede || ' ' || nm_bairro) AS nm_mercado,
        (nm_produto || ' ' || nm_tipo) AS nm_produto,
        first(vl_preco_unitario * vl_quantidade) OVER (PARTITION BY nm_pesquisa ORDER BY rank ASC) AS vl_menor_preco,
        (vl_preco_unitario * vl_quantidade) AS vl_preco_total
    FROM mais_baratos

""")

display(df_basket_result.groupBy('nm_pesquisa', 'nm_produto', 'vl_menor_preco').pivot('nm_mercado').sum('vl_preco_total'))
